Step 0. Unzip enron1.zip into the current directory.

Step 1. Traverse the dataset and create a Pandas dataframe. This is already done for you and should run without any errors. You should recognize Pandas from task 1.

In [9]:
import pandas as pd
import os

def read_spam():
    category = 'spam'
    directory = './enron1/spam'
    return read_category(category, directory)

def read_ham():
    category = 'ham'
    directory = './enron1/ham'
    return read_category(category, directory)

def read_category(category, directory):
    emails = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        with open(os.path.join(directory, filename), 'r') as fp:
            try:
                content = fp.read()
                emails.append({'name': filename, 'content': content, 'category': category})
            except:
                print(f'skipped {filename}')
    return emails

ham = read_ham()
spam = read_spam()

ham_df = pd.DataFrame.from_records(ham)
spam_df = pd.DataFrame.from_records(spam)

print("Number of Ham Emails:", len(ham_df))
print("Number of Spam Emails:", len(spam_df))


skipped 2649.2004-10-27.GP.spam.txt
skipped 0754.2004-04-01.GP.spam.txt
skipped 2042.2004-08-30.GP.spam.txt
skipped 3304.2004-12-26.GP.spam.txt
skipped 4142.2005-03-31.GP.spam.txt
skipped 3364.2005-01-01.GP.spam.txt
skipped 4201.2005-04-05.GP.spam.txt
skipped 2140.2004-09-13.GP.spam.txt
skipped 2248.2004-09-23.GP.spam.txt
skipped 4350.2005-04-23.GP.spam.txt
skipped 4566.2005-05-24.GP.spam.txt
skipped 2526.2004-10-17.GP.spam.txt
skipped 1414.2004-06-24.GP.spam.txt
skipped 2698.2004-10-31.GP.spam.txt
skipped 5105.2005-08-31.GP.spam.txt
Number of Ham Emails: 3672
Number of Spam Emails: 1485


Step 2. Data cleaning is a critical part of machine learning. You and I can recognize that 'Hello' and 'hello' are the same word but a machine does not know this a priori. Therefore, we can 'help' the machine by conducting such normalization steps for it. Write a function `preprocessor` that takes in a string and replaces all non alphabet characters with a space and then lowercases the result.

In [13]:
import re

def preprocessor(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    return text

Step 3. We will now train the machine learning model. All the functions that you will need are imported for you. The instructions explain how the work and hint at which functions to use. You will likely need to refer to the scikit learn documentation to see how exactly to invoke the functions. It will be handy to keep that tab open.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# The CountVectorizer converts a text sample into a vector (think of it as an array of floats).
# Each entry in the vector corresponds to a single word and the value is the number of times the word appeared.
# Instantiate a CountVectorizer. Make sure to include the preprocessor you previously wrote in the constructor.


vectorizer = CountVectorizer(preprocessor=preprocessor)


# Use train_test_split to split the dataset into a train dataset and a test dataset.
# The machine learning model learns from the train dataset.
# Then the trained model is tested on the test dataset to see if it actually learned anything.
# If it just memorized for example, then it would have a low accuracy on the test dataset and a high accuracy on the train dataset.


# Split the ham dataset
X_train_ham, X_test_ham, y_train_ham, y_test_ham = train_test_split(ham_df["content"], ham_df["category"], test_size=0.2, random_state=1)

# Split the spam dataset
X_train_spam, X_test_spam, y_train_spam, y_test_spam = train_test_split(spam_df["content"], spam_df["category"], test_size=0.2, random_state=1)

# Combine the ham and spam train datasets
X_train = pd.concat([X_train_ham, X_train_spam])
y_train = pd.concat([y_train_ham, y_train_spam])

# Combine the ham and spam test datasets
X_test = pd.concat([X_test_ham, X_test_spam])
y_test = pd.concat([y_test_ham, y_test_spam])



# Fit the vectorizer on the training data and transform it
X_train_vectorized = vectorizer.fit_transform(X_train)

# Transform the test data using the same vectorizer
X_test_vectorized = vectorizer.transform(X_test)



# Use the LogisticRegression model to fit to the train dataset.
# You may remember y = mx + b and Linear Regression from high school. Here, we fitted a scatter plot to a line.
# Logistic Regression is another form of regression. 
# However, Logistic Regression helps us determine if a point should be in category A or B, which is a perfect fit.

# Instantiate the Logistic Regression model
model = LogisticRegression()

# Fit the model on the vectorized training data
model.fit(X_train_vectorized, y_train)



# Validate that the model has learned something.
# Recall the model operates on vectors. First transform the test set using the vectorizer. 
# Then generate the predictions.

# Predict the categories for the test data
y_pred = model.predict(X_test_vectorized)


# Calculate and print the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate and print the confusion matrix
confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion)

# Generate and print the classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)


# We now want to see how we have done. We will be using three functions.
# `accuracy_score` tells us how well we have done. 
# 90% means that every 9 of 10 entries from the test dataset were predicted accurately.
# The `confusion_matrix` is a 2x2 matrix that gives us more insight.
# The top left shows us how many ham emails were predicted to be ham (that's good!).
# The bottom right shows us how many spam emails were predicted to be spam (that's good!).
# The other two quadrants tell us the misclassifications.
# Finally, the `classification_report` gives us detailed statistics which you may have seen in a statistics class.
# TODO



Accuracy: 0.9738372093023255
Confusion Matrix:
 [[715  20]
 [  7 290]]
Classification Report:
               precision    recall  f1-score   support

         ham       0.99      0.97      0.98       735
        spam       0.94      0.98      0.96       297

    accuracy                           0.97      1032
   macro avg       0.96      0.97      0.97      1032
weighted avg       0.97      0.97      0.97      1032



Step 4.

In [30]:
# Let's see which features (aka columns) the vectorizer created. 
# They should be all the words that were contained in the training dataset.

feature_names = vectorizer.get_feature_names_out()

# You may be wondering what a machine learning model is tangibly. It is just a collection of numbers. 
# You can access these numbers known as "coefficients" from the coef_ property of the model
# We will be looking at coef_[0] which represents the importance of each feature.
# What does importance mean in this context?
# Some words are more important than others for the model.
# It's nothing personal, just that spam emails tend to contain some words more frequently.
# This indicates to the model that having that word would make a new email more likely to be spam.
# TODO

coefficients = model.coef_[0]

# Iterate over importance and find the top 10 positive features with the largest magnitude.
# Similarly, find the top 10 negative features with the largest magnitude.
# Positive features correspond to spam. Negative features correspond to ham.
# You will see that `http` is the strongest feature that corresponds to spam emails. 
# It makes sense. Spam emails often want you to click on a link.
# TODO


features_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# Get top 10 positive and negative features directly
top_positive_features = features_df.nlargest(10, 'Coefficient')
top_negative_features = features_df.nsmallest(10, 'Coefficient')

print("Top 10 Positive Features (Spam Indicators):")
print(top_positive_features[['Feature', 'Coefficient']])

print("\nTop 10 Negative Features (Ham Indicators):")
print(top_negative_features[['Feature', 'Coefficient']])


Top 10 Positive Features (Spam Indicators):
        Feature  Coefficient
17755      http     1.035356
28410    prices     0.790043
25128        no     0.754002
26540  paliourg     0.746452
30229   removed     0.725002
16994     hello     0.709924
24109      more     0.704826
24012     money     0.689934
26506      pain     0.681742
30228    remove     0.679168

Top 10 Negative Features (Ham Indicators):
        Feature  Coefficient
2548   attached    -1.703950
12616     enron    -1.609054
35453    thanks    -1.464468
9493      daren    -1.383290
11057       doc    -1.259404
9685       deal    -1.175829
23384     meter    -1.174393
39478       xls    -1.129080
32891    sitara    -1.069464
24826      neon    -1.059852


Submission
1. Upload the jupyter notebook to Forage.

All Done!